### TODO per Valentina

- go through the files she sent on Teams
- think once again whats how to improve actions identification - still does mistakes
- Make possible to query two files at once
- get some information on where the fuck does she want to publish?


In [1]:
from llm import load_models
from dotenv import load_dotenv
from questioning import load_qdf, ask_qdfs, pdfs_to_documents, documents_to_vector_store, query_action_detail, query_action_SMART, query_action_list, ask_RAG, filter_action_pages

from logging_config import logging_config
logger = logging_config(f'GPT@JRC','logger.log')
import pandas as pd
from tqdm import tqdm
from prompts import PROMPTS
from pydantic import BaseModel, Field
from typing import List, Literal, Optional

config = {
    'Torino':{
        'pdfs': [
            {'file':'Torino.pdf','action_page_limit': (143, 191)}
        ],
    },
    'Zaragoza':{
        'pdfs': [
            {'file':'Zaragoza.pdf','action_page_limit': (0, 200)}
        ],
    },
    'Bologna':{
        'pdfs': [
            {'file':'Bologna.pdf','action_page_limit': (128, 194)}
        ],
    },
    'Izmir':{
        'pdfs': [
            {'file':'Izmir.pdf','action_page_limit': (58, 142)}
        ],
    },
    'Lisbon':{
        'pdfs': [
            {'file':'Lisbon_1.pdf','action_page_limit': (0, 200)},
            {'file':'Lisbon_2.pdf','action_page_limit': (0, 200)}
        ],
    },
    'Liepaja':{
        'pdfs': [
            {'file':'Liepaja_1.pdf','action_page_limit': (60, 100)},
            #{'file':'Liepaja_2.pdf','action_page_limit': (0, 200)} # seem to be old version
        ],
    },
    'Valencia':{
        'pdfs': [
            {'file':'Valencia_1.pdf','action_page_limit': (0, 200)},
            {'file':'Valencia_2.pdf','action_page_limit': (0, 200)}
        ],
    },
    'Porto':{
        'pdfs': [
            {'file':'Porto_1.pdf','action_page_limit': (0, 200)},
            {'file':'Porto_2.pdf','action_page_limit': (0, 200)}
        ],
    },
    'Lappeenranta':{
        'pdfs': [
            {'file':'Lappeenranta.pdf','action_page_limit': (0, 200)}
            
        ],
    },
    'Firenze':{
        'pdfs': [
            {'file':'Firenze.pdf','action_page_limit': (0, 200)},
        ],
    },
    'Ioannina':{
        'pdfs': [
            {'file':'Ioannina.pdf','action_page_limit': (0, 200)}
        ]
    },
    'Kalamata':{
        'pdfs': [
            {'file':'Kalamata.pdf','action_page_limit': (0, 200)}
        ]
    },
    'Tampere':{
        'pdfs': [
            {'file':'Tampere.pdf','action_page_limit': (0, 200)}
        ],
    },
    'Espoo':{
        'pdfs': [
            {'file':'Espoo.pdf','action_page_limit': (0, 200)}
        ],
    },
    'Guimaraes':{
        'pdfs': [
            {'file':'Guimaraes.pdf','action_page_limit': (0, 200)}
        ]
    },
    'Lahti':{
        'pdfs': [
            {'file':'Lahti.pdf','action_page_limit': (0, 200)}
        ],
    },
    'Kozani':{
        'pdfs': [
            {'file':'Kozani.pdf','action_page_limit': (0, 200)}
        ],
    },
    'Pecs':{
        'pdfs': [
            {'file':'Pecs.pdf','action_page_limit': (0, 200)}
        ],
    },
    'Sevilla':{
        'pdfs': [
            {'file':'Sevilla.pdf','action_page_limit': (0, 200)}
        ],
    },
}

load_dotenv()
llm, emb = load_models(service = 'DeepInfra')
qdf = load_qdf() # questions from JSON
pdf_prefix='pdf_input/'

In [2]:
def pdf_to_excel(city, excel_output, llm=llm, emb=emb, pdf_prefix='pdf_input/',config=config):
    files = [f'{pdf_prefix}{pdf["file"]}' for pdf in config[city]['pdfs']]

    logger.info(f'Starting PDFs of {city}')    
    logger.debug('Reading the documents..')
    documents, splits = pdfs_to_documents(files)
    
    logger.debug('Converting to vector store ...')
    vector_store = documents_to_vector_store(documents, emb)

    with pd.ExcelWriter(excel_output) as xlsx:

        qs_responses = ask_qdfs(
            qdf = qdf,
            llm = llm, 
            vector_store = vector_store,
            city = city,
            logger=logger
        )

        qs_responses.to_excel(xlsx,sheet_name='standard_qs')

        logger.debug('Barriers...')

        class Barrier(BaseModel):
            title: str
            brief_description: str
            page_reference: str
            explanation: str
            category: List[Literal["Leadership", "Financial", "Regulatory", "Operational", "Organisational", "Partnerships", "Social", "Environmental", "Safety and Security", "Other"]]
        
        class BarrierList(BaseModel):
            barriers: List[Barrier]


        response = ask_RAG(
            embed_query = 'List all barriers to accelerating emission reduction in view of the 2030 climate-neutrality goal that are explicitly described in the plan',
            vector_store = vector_store,
            response_model=BarrierList,
            llm = llm,
            logger = logger,
            template_key = 'barriers_prompt',
            template_kwargs={},
            city = city
        )
        bars = pd.DataFrame([b for b in response['barriers']])
        bars.to_excel(xlsx, sheet_name = 'barriers')

        logger.debug('Participatory processes...')

        class Stakeholder(BaseModel):
            stakeholder: str
            role: str

        class ParticipatoryProcess(BaseModel):
            title: str
            brief_description: str = Field(description="Description in English")
            page_reference: str
            explanation: str
            relevant_stakeholders: List[Stakeholder]
            implementation_barriers: Optional[List[str]] = []
            barriers_solution: Optional[List[str]] = []
            citizen_contribution_clearly_defined: bool

        class ParticipatoryProcesses(BaseModel):
            participatory_processes: List[ParticipatoryProcess]


        response = ask_RAG(
            embed_query='''
            - List all participatory process described in the SECAP plan.
            - Describe how citizens' contribute to specific priorities in the plan
            - Inclusion of participatory processes and engagement of stakeholder
            - Barriers to /challenges for citizen engagement and solutions/opportunities to remove it
            ''',
            template_key='participatory_processes',
            template_kwargs={},
            response_model=ParticipatoryProcesses,
            vector_store=vector_store,
            llm=llm,
            logger=logger,
            city=city
        )

        participatory_processes = pd.DataFrame([pp for pp in response['participatory_processes']])
        participatory_processes.to_excel(xlsx, sheet_name='participatory_processes')

        logger.info(f'Action list...')

        action_list = query_action_list(
            llm=llm, 
            documents=filter_action_pages(splits, config[city]),
            logger=logger
        )

        logger.info(f'Action details...')
        ads = [query_action_detail(
                action=action, 
                llm=llm, 
                vector_store=vector_store, 
                logger=logger,
                city = city
            ) 
            for action in tqdm(action_list.to_dict(orient='records'),desc='Action details')]
        action_details = pd.json_normalize([ad for ad in ads if ad is not None])

        action_details.to_excel(xlsx, sheet_name='Actions')
        logger.info(f'Action SMARTs...')

        smarts = [query_action_SMART(
                action=action,
                llm=llm,
                vector_store=vector_store, 
                logger=logger,
                city = city
            ) for action in tqdm(action_list.to_dict(orient='records'),desc='Action SMART')]
        action_SMART = pd.json_normalize([sm for sm in smarts if sm is not None])

        action_SMART.to_excel(xlsx, sheet_name='Actions_SMART')

for city in list(config.keys())[-11:]:
    pdf_to_excel(
        city, 
        f'output/{city}.xlsx')

2025-01-15 13:01:00,979 - GPT@JRC - INFO - Starting PDFs of Lappeenranta


Questions: 100%|██████████| 58/58 [09:03<00:00,  9.36s/it]


2025-01-15 13:11:17,299 - GPT@JRC - INFO - Action list...
2025-01-15 13:12:46,674 - GPT@JRC - INFO - Action details...


Action details: 100%|██████████| 22/22 [06:56<00:00, 18.92s/it]

2025-01-15 13:19:42,869 - GPT@JRC - INFO - Action SMARTs...



Action SMART:  59%|█████▉    | 13/22 [01:42<01:05,  7.33s/it]